In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
# import os

# os.environ['SPARK_HOME'] = '/Users/reidbongard/apps/spark-3.5.0-bin-hadoop3'

In [3]:
# # os.environ['SPARK_HOME']
# os.environ['PYTHONPATH']

# Q1

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 12:42:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark.version

'3.5.0'

# Q2

In [6]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz -O fhvhv_tripdata_2019-10.csv.gz
# ! gzip -d fhvhv_tripdata_2019-10.csv.gz

In [7]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2019-10.csv')

In [8]:
df.rdd.getNumPartitions()

8

In [9]:
df = df.coalesce(1)

In [10]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [11]:
df = df.repartition(6)

In [12]:
df.rdd.getNumPartitions()

6

In [13]:
df.write.parquet('fhvhv/2019/10/', mode="overwrite")

In [14]:
! ls -l fhvhv/2019/10/

total 74496
-rw-r--r--  1 reidbongard  staff        0 Mar  4 12:42 _SUCCESS
-rw-r--r--  1 reidbongard  staff  6202128 Mar  4 12:42 part-00000-c35704b7-8c35-42fa-87bd-f4a6b134efc7-c000.snappy.parquet
-rw-r--r--  1 reidbongard  staff  6202647 Mar  4 12:42 part-00001-c35704b7-8c35-42fa-87bd-f4a6b134efc7-c000.snappy.parquet
-rw-r--r--  1 reidbongard  staff  6211938 Mar  4 12:42 part-00002-c35704b7-8c35-42fa-87bd-f4a6b134efc7-c000.snappy.parquet
-rw-r--r--  1 reidbongard  staff  6237944 Mar  4 12:42 part-00003-c35704b7-8c35-42fa-87bd-f4a6b134efc7-c000.snappy.parquet
-rw-r--r--  1 reidbongard  staff  6203575 Mar  4 12:42 part-00004-c35704b7-8c35-42fa-87bd-f4a6b134efc7-c000.snappy.parquet
-rw-r--r--  1 reidbongard  staff  6198001 Mar  4 12:42 part-00005-c35704b7-8c35-42fa-87bd-f4a6b134efc7-c000.snappy.parquet


In [15]:
# ! rm -r fhvhv/2019/10/

In [16]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [17]:
from pyspark.sql import SparkSession

In [18]:
df.registerTempTable('trips_data')

/Users/reidbongard/anaconda3/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


# Q3

In [19]:
spark.sql("""
SELECT
    COUNT(*)
FROM
    trips_data
WHERE TO_DATE(pickup_datetime) = '2019-10-15'

""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



# Q4

In [20]:
spark.sql("""
SELECT
    MAX(DATEDIFF(hour, t.pickup_datetime, t.dropOff_datetime)) max_trip_time
FROM
    trips_data t
""").show()

+-------------+
|max_trip_time|
+-------------+
|       631152|
+-------------+



# Q6

In [21]:
# ! wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

In [22]:
# ! ls

In [23]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [24]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [25]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01239|2019-10-04 12:15:42|2019-10-04 12:42:54|         264|         247|   NULL|                B03127|
|              B00272|2019-10-02 17:30:00|2019-10-02 18:25:00|         264|         264|   NULL|                B00272|
|              B01057|2019-10-02 11:00:00|2019-10-02 11:20:00|         264|         264|   NULL|                B01057|
|              B02735|2019-10-02 10:57:22|2019-10-02 11:28:59|         264|          20|   NULL|                B02878|
|              B01362|2019-10-04 11:49:56|2019-10-04 11:58:13|         264|         159|   NULL|                B01362|
|              B01800|2019-10-02 15:35:0

In [26]:
df_zones_tmp = df_zones.\
               withColumnRenamed('LocationID', 'PULocationID').\
               withColumnRenamed('Borough', 'PUBorough'). \
               withColumnRenamed('Zone', 'PUZone')

In [27]:
df_joined = df.join(df_zones_tmp, on='PULocationID', how='left')

In [28]:
df_joined.head(1)

[Row(PUlocationID='264', dispatching_base_num='B01239', pickup_datetime='2019-10-04 12:15:42', dropOff_datetime='2019-10-04 12:42:54', DOlocationID='247', SR_Flag=None, Affiliated_base_number='B03127', PUBorough='Unknown', PUZone='NV', service_zone='N/A')]

In [29]:
df_joined.registerTempTable('df_joined')

In [30]:
spark.sql("""
SELECT
    PUZone,
    count(1) pickup_count
FROM
    df_joined
WHERE MONTH(pickup_datetime) = 10
GROUP BY PUZone
ORDER BY pickup_count

""").show()

+--------------------+------------+
|              PUZone|pickup_count|
+--------------------+------------+
|         Jamaica Bay|           1|
|Governor's Island...|           2|
| Green-Wood Cemetery|           5|
|       Broad Channel|           8|
|     Highbridge Park|          14|
|        Battery Park|          15|
|Saint Michaels Ce...|          23|
|Breezy Point/Fort...|          25|
|Marine Park/Floyd...|          26|
|        Astoria Park|          29|
|    Inwood Hill Park|          39|
|       Willets Point|          47|
|Forest Park/Highl...|          53|
|  Brooklyn Navy Yard|          57|
|        Crotona Park|          62|
|        Country Club|          77|
|     Freshkills Park|          89|
|       Prospect Park|          98|
|     Columbia Street|         105|
|  South Williamsburg|         110|
+--------------------+------------+
only showing top 20 rows

